# Structures `kissim` cannot read

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from pathlib import Path

import pandas as pd
from opencadd.databases.klifs import setup_remote, setup_local

from kissim.encoding import Fingerprint, FingerprintGenerator

In [3]:
logger_opencadd = logging.getLogger("opencadd")
logger_opencadd.setLevel(logging.INFO)
formatter = logging.Formatter(logging.BASIC_FORMAT)
s_handler = logging.StreamHandler()
s_handler.setFormatter(formatter)
logger_opencadd.addHandler(s_handler)

## Extract new structures

In [4]:
klifs_ids1 = pd.read_csv(
    "https://github.com/volkamerlab/opencadd/raw/master/opencadd/data/klifs_ids.20210115.csv.gz"
)
klifs_ids2 = pd.read_csv(
    "https://github.com/volkamerlab/opencadd/raw/master/opencadd/data/klifs_ids.20210701.csv.gz"
)

In [5]:
diff = list(
    set(klifs_ids2["structure.klifs_id"].to_list())
    - set(klifs_ids1["structure.klifs_id"].to_list())
)

In [6]:
print(f"Number of new structures: {len(diff)}")

Number of new structures: 531


In [7]:
print("KLIFS ID numbering consecutive?")
len(diff) == (diff[-1] - diff[0] + 1)

KLIFS ID numbering consecutive?


True

In [8]:
print(f"Structures with IDs from {diff[0]} are new.")

Structures with IDs from 13232 are new.


## Problematic structures

In [9]:
structure_klifs_ids_io = [1243, 13623, 13625, 13626, 13624]
structure_klifs_ids_encoding = [13556]
structure_klifs_ids = structure_klifs_ids_io + structure_klifs_ids_encoding

Why is 1243 a problem now?

In [10]:
klifs_ids1[klifs_ids1["structure.klifs_id"] == 1243]

,structure.klifs_id,structure.pdb_id,structure.alternate_model,structure.chain,kinase.klifs_name,kinase.klifs_id,ligand.expo_id
1219,1243,2ogv,-,A,PDGFRb,453,-


In [11]:
klifs_ids2[klifs_ids2["structure.klifs_id"] == 1243]

,structure.klifs_id,structure.pdb_id,structure.alternate_model,structure.chain,kinase.klifs_name,kinase.klifs_id,ligand.expo_id
1219,1243,2ogv,-,A,FMS,449,-


## Reproduce errors

In [12]:
local = setup_local(
    "/home/dominique/Documents/GitHub/kissim_app/data/external/structures/20210630_KLIFS_HUMAN/"
)

INFO:opencadd.databases.klifs.api:Set up local session...
INFO:opencadd.databases.klifs.local:Load overview.csv...
INFO:opencadd.databases.klifs.local:Load KLIFS_export.csv...
INFO:opencadd.databases.klifs.local:Merge both csv files...
INFO:opencadd.databases.klifs.local:Add paths to coordinate folders to structures...
INFO:opencadd.databases.klifs.local:Add KLIFS IDs to structures (uses remote since not available locally!)...
INFO:opencadd.databases.klifs.api:Local session is ready!


In [13]:
try:
    local.pockets.by_structure_klifs_id(1243, "pdb")
except ValueError as e:
    print(e)

Number of KLIFS pocket residues are not equal inpocket sequence string (83) and pocket structure file (85).


In [14]:
try:
    local.pockets.by_structure_klifs_id(13623, "pdb")
except ValueError as e:
    print(e)

Number of KLIFS pocket sequence is 57 but must be 85.


In [15]:
f_local = FingerprintGenerator.from_structure_klifs_ids(
    [13556], klifs_session=local
)

INFO:opencadd.structure.pocket.base:Pocket None (set pocket residues): The following input residues PDB IDs were assigned to the value None because they cannot be cast to an integer (residue PDB ID, residue index): [('_', 6), ('_', 7), ('_', 8), ('_', 9), ('_', 83), ('_', 84), ('_', 85)]
INFO:opencadd.structure.pocket.base:Pocket 13556 (set pocket residues): The following input residues PDB IDs were assigned to the value None because they cannot be cast to an integer (residue PDB ID, residue index): [(None, 6), (None, 7), (None, 8), (None, 9), (None, 83), (None, 84), (None, 85)]
INFO:opencadd.structure.pocket.core:Pocket 13556: Missing pocket CA atoms. The pocket center is calculated based on 79 CA atoms (total number of pocket residues is 85).
Fingerprint generation throw error for 13556:  '2 residues were found, but must be 1 or 0. Duplicated residue serial number(s): [621]'
